In [1]:
import os
import pandas as pd
from functions import get_all_origins, find_pattern_for_quantity, convert_to_grams, relation_qnt_preco, remove_spaces, clean_text
import re
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
df = get_all_origins()

In [5]:
import re

# Verifique se cada valor da coluna 'especificacao_rota' é uma string
filtro = df['titulo'].apply(lambda x: isinstance(x, str) and bool(re.search(r'(\d+([.,]\d+)?)\s*(kg|g|gr|gramas)\s*(\w+)', x)))

# Use o filtro para selecionar as linhas relevantes do DataFrame
resultado = df[filtro][['titulo', 'quantidade']]
resultado


,titulo,quantidade
64,combo 2x kimera extreme 2x creatina 90g porta ...,NaN
66,combo kimera ultimate creatina 90g porta caps,NaN
88,melatonina 1000 gotas 021 mg por gota,NaN
105,best vegan rtd 15g protein chocolate,NaN
106,best vegan rtd 15g protein morango com leite c...,NaN
129,best whey bar 12g torta al limone,NaN
130,best whey bar 12g pao de mel,NaN
132,best whey bar 12g duo bianco al latte,NaN
135,best whey bar 12g cheesecake de maracuja,NaN
136,best whey bar 12g cookies cream,NaN


In [11]:

df = df[df['ref'] == "9085321c"]
pattern = r'(\d+[.,]?\d*)\s*(kg|g|gr|gramas)(?!\s*gratis)\b'
df[['quantidade', 'unidade']] = df['titulo'].apply(lambda text: find_pattern_for_quantity(text, pattern)).apply(pd.Series)
df['quantidade'] = df[['quantidade', 'unidade']].apply(convert_to_grams, axis=1)
df['preco_qnt'] = df.apply(relation_qnt_preco, axis=1)
df['quantidade'] = df['quantidade'].astype(str).replace("-1", np.nan)
df

# pattern = r'(\d+)\s*(caps|cap|cáps|v-caps|cápsulas|capsules|cáp|comprimidos|comps|comp|capsulas|barras|soft|softgel|sachês|doses|dose|tabs|tabletes|tablets|gotas)\b'
# df[['quantidade_formato', 'formato']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)
# df['formato'] = df['formato'].apply(substituir_por_comprimidos)

# pattern = r'(\d+)\s*(caixa|display\s*c/|unidades|unid|un|saches|caixas|x)\b'
# df[['quantidade_formato_2', 'formato_2']] = df['nome'].apply(lambda text: mask_qnt(text, pattern)).apply(pd.Series)

,ref,titulo,preco,link_imagem,link_produto,ing_date,nome,marca,preco_numeric,quantidade,preco_qnt,quantidade_formato,formato,especificacao,especificacao_rota,unidade
305,9085321c,tasty iso 1800g bag,"407,55",https://adaptogen.com.br/wp-content/uploads/20...,https://adaptogen.com.br/tasty-iso-1800g-bag/,17/11/2023,"tasty iso 1,800g bag",adaptogen,407.55,1800,0.226,NaN,NaN,NaN,whey protein iso,g
